In [7]:
import pandas as pd
import numpy as np

In [8]:
ratings = np.load('small_ratings.npy') # 624, 3 each list has 3 values: userid,movieid,rating
users = np.load('users.npy')[:1000] # 1000 users
movies = np.load('movies.npy')[:1000] # 1000 movies

In [69]:
factors = 5 #num of factors to be used in matrix factorization
iterations = 100000
num_user = len(users)
num_movies = len(movies)
num_ratings = len(ratings)
movie_meta_data = pd.read_csv('movies_metadata.csv')
iterations = 100000
lr = 1e-3
lambda_p=0.02

C:\Users\TR\AppData\Local\Temp\ipykernel_14640\2581442450.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_meta_data = pd.read_csv('movies_metadata.csv')


In [10]:
# Scale and normalize ratings 
ratings[:, 2] -= 2.5
ratings[:, 2] /= 2.5

In [11]:
#Initialize the matrices
W = np.random.uniform(low=-0.5, high=0.5, size= (num_user, factors))
H = np.random.uniform(low=-0.5, high=0.5, size= (num_movies, factors))

In [12]:
D_s = []
for ui, mi, ri in ratings:
    for uj, mj, rj in ratings:
        if ui != uj or mi == mj:
            continue
        if ri > rj:
            D_s.append((int(ui),int(mi),int(mj)))
    
D_s = list(set(D_s)) # gert rid of the duplicates

In [13]:
def former_part(x_uij, partial_x):
    exp_x = np.exp(-x_uij)
    out = exp_x / (1 + exp_x) * partial_x
    return out

In [70]:
#Training 

for itr in range(iterations):
    u, i, j = D_s[np.random.randint(len(D_s))]
    
    xui = np.dot(W[u],H[i])
    xuj = np.dot(W[u],H[j])
    
    diff = xui - xuj
    
    for f in range(factors):
        W[u,f] -= lr * ( former_part(diff, H[i,f] - H[j,f] ) + lambda_p * W[u,f])
        H[i,f] -= lr * ( former_part(diff, W[u,f]) + lambda_p * H[i,f])
        H[j,f] -= lr * ( former_part(diff, -W[u,f]) + lambda_p * H[j,f])
        
    if itr % 10000 == 0:
        print("Tracking the changes in the parameters:")
        print(W[25,1])
        #print(H[10,1])

Changes in the parameters:
0.37050953773075956
Changes in the parameters:
0.36956542388138336
Changes in the parameters:
0.3689585648956101
Changes in the parameters:
0.36832971165382306
Changes in the parameters:
0.36832971165382306
Changes in the parameters:
0.36719185844552604
Changes in the parameters:
0.3660230488430287
Changes in the parameters:
0.3655749147688197
Changes in the parameters:
0.3643023543748173
Changes in the parameters:
0.36386245950960927


In [15]:
selected_user_id = 19

In [16]:
# Predict our matrix
X_hat = np.dot(W,H.T)

In [41]:
# Select top 5 suggestions
selected_movies = np.argsort(X_hat[selected_user_id,])[:5]

In [65]:
for i, j in enumerate(selected_movies):
    movie_title = movie_meta_data.iloc[j,]["original_title"]
    genre = movie_meta_data.iloc[j,]["genres"]
    print(f"Suggestion number {i}")
    print(f"Title: {movie_title}")
    print(f"ID & Genre: {genre} ")
    print("-------------------")

Suggestion number 0
Title: Vampire in Brooklyn
ID & Genre: [{'id': 35, 'name': 'Comedy'}, {'id': 27, 'name': 'Horror'}, {'id': 10749, 'name': 'Romance'}] 
-------------------
Suggestion number 1
Title: Angel and the Badman
ID & Genre: [{'id': 10749, 'name': 'Romance'}, {'id': 37, 'name': 'Western'}] 
-------------------
Suggestion number 2
Title: Tank Girl
ID & Genre: [{'id': 28, 'name': 'Action'}, {'id': 35, 'name': 'Comedy'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}] 
-------------------
Suggestion number 3
Title: August
ID & Genre: [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}] 
-------------------
Suggestion number 4
Title: The Fugitive
ID & Genre: [{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}, {'id': 80, 'name': 'Crime'}, {'id': 9648, 'name': 'Mystery'}] 
-------------------
